# HE prediction
This tutorial shows how to construct a testing workflow of HE prediction task.
- Data: ATACH-2 + Yale
- Train data: Trained 507 patients (HE=83) and validated 127 patients (HE=21).
- Test data: 159 patients
- AUC = 0.79

And it contains below features:
- CT input with thick thickness
- Pre-processing: extract brain window, skull stripping, remove boundary, resample
- Segmentation and Prediction: 3D SegResNet model, DenseNet.


## Setup environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/bkHEprediction")
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2023-06-14 14:59:54.948732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 14:59:57.185959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Setup data directory

You can specify a directory with variable.
This allows you to save results and reuse for the next step.

## Setup function for segmentation
**getbrainwindow(nii)** extract the brain window from CT with window_center=40, window_width=80
- Input: nii which load the CT
- Output: numpy array image [0,1]

**removeSkull(img1)** skull stripping
- Input: numpy array image
- Output: brain region (numpy array image [0,1])

**getDilation(img1, img1seg)** return dilation of objects inside image

**resizeImgResizeWithPadOrCrop(img1, size):** return image with pad or crop to the size

## Setup path variable


In [3]:
import monai
from monai.data import  DataLoader, ImageDataset
from monai.transforms import (
    Resize, NormalizeIntensity, Activations, Compose, EnsureType, CenterSpatialCrop,ScaleIntensity,ResizeWithPadOrCrop, ResizeWithPadOrCropd,
    LoadImaged, EnsureChannelFirstd, EnsureTyped, NormalizeIntensityd, ScaleIntensityd,AddChannel
)
from monai.data import CacheDataset, DataLoader, ImageDataset, Dataset
from monai.data import decollate_batch
from monai.networks.nets import DenseNet121
from skimage.morphology import disk, binary_dilation, binary_erosion, remove_small_objects
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import scipy.ndimage as nd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib
import nibabel.processing
from monai.utils import set_determinism
import random
import scipy
random.seed(123)
set_determinism(seed=123)
from monai.metrics import DiceMetric
from monai.networks.nets import SegResNet
from monai.inferers import sliding_window_inference
from sklearn.metrics import (
    classification_report, confusion_matrix,
    ConfusionMatrixDisplay
)
VAL_AMP = True
def inference(input):
    def _compute(input):
        return sliding_window_inference(
            inputs=input,
            roi_size=(256, 256, 32),
            sw_batch_size=4,
            predictor=model,
            overlap=0.5,
        )

    if VAL_AMP:
        with torch.cuda.amp.autocast():
            return _compute(input)
    else:
        return _compute(input)
from monai.transforms import (
    Activations, AsDiscrete,
    Compose
)
import warnings
warnings.filterwarnings('ignore')
import time
import os.path
from os import path
X=0
Y=1
Z=2
def resample3d(image, spacing, new_spacing):
    resize_x = spacing[X] / new_spacing[X]
    new_shape_x = np.round(image.shape[X] * resize_x)
    resize_x = float(new_shape_x) / float(image.shape[X])
    sx = spacing[X] / resize_x

    resize_y = spacing[Y] / new_spacing[Y]
    new_shape_y = np.round(image.shape[Y] * resize_y)
    resize_y = new_shape_y / image.shape[Y]
    sy = spacing[Y] / resize_y

    resize_z = spacing[Z] / new_spacing[Z]
    new_shape_z = np.round(image.shape[Z] * resize_z)
    resize_z = float(new_shape_z) / float(image.shape[Z])
    sz = spacing[Z] / resize_z

    image = scipy.ndimage.interpolation.zoom(image, (resize_x, resize_y, resize_z), order=1)

    return image


def getbrainwindow(nii):
    window_center, window_width = 40, 80
    tmpimg1 = nii.get_fdata()
    tmpimg1[tmpimg1 < 0] = 0
    tmpimg1[tmpimg1 > 200] = 0
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    tmpimg1[tmpimg1 < img_min] = img_min
    tmpimg1[tmpimg1 > img_max] = img_max
    tmpimg1 = (tmpimg1 - tmpimg1.min()) / np.ptp(tmpimg1)
    return tmpimg1

def removeSkull(img1):
    img_bw = img1.copy()
    img_bw[img_bw > 0] = 1
    for slice in range(0, img_bw.shape[2]):
        if slice < round(img_bw.shape[2] / 7) or slice > (img_bw.shape[2] - round(img_bw.shape[2] / 7)):
            img_bw[:, :, slice] = 0
        if img_bw[:, :, slice].sum() > 0:
            img_bw[:, :, slice] = binary_erosion(img_bw[:, :, slice].astype(np.uint8),
                                                 disk(4, dtype=bool))
            img_bw[:, :, slice] = remove_small_objects(img_bw[:, :, slice].astype(bool), 5000)
            img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                  disk(4, dtype=bool))
            img_bw[:, :, slice] = nd.binary_fill_holes(img_bw[:, :, slice].astype(np.uint8))
        if img_bw[:, :, slice].sum() > 0:
            contours, _ = cv2.findContours(img_bw[:, :, slice].astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            cv2.drawContours(mask, contours, -1, (255, 255, 255), -1)
            img_bw[:, :, slice][mask == 0] = 0
            mask = np.zeros(img_bw[:, :, slice].shape, dtype=np.uint8)
            cv2.drawContours(mask, contours, -1, (255, 255, 255), 20)
            img_bw[:, :, slice][mask == 255] = 0
    img1[img_bw == 0] = 0
    img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)
    return img1

def getDilation(img1, img1seg):
    if img1seg.sum() <= 0:
        return img1seg
    img1dilation = img1.copy()
    img_bw = img1seg.copy()
    for slice in range(0, img_bw.shape[2]):
        if img_bw[:, :, slice].sum() > 0:
            img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                  disk(10, dtype=bool))
    img1dilation[img_bw == 0] = 0
    img1dilation = 1.0 * (img1dilation - img1dilation.min()) / np.ptp(img1dilation)
    return img1dilation


def resizeImgResizeWithPadOrCrop(img1, size):
    resizeImage = ResizeWithPadOrCrop(spatial_size=size)
    image = np.zeros([1, img1.shape[0], img1.shape[1], img1.shape[2]])
    image[0, :, :, :] = img1
    image = resizeImage(image)
    return image
data_dir = "italy_ds/"
baseline_path = "baseline"
output_path = data_dir + "preprocessing"
output_path_pre_classification = data_dir + "classification"
raw_data = pd.read_csv(os.path.join(data_dir, "labels.csv"), sep=";")
weight_segmentation_path = "full_28_3_original_segmentation3d_1_1.pth"
weight_classification_path = "6000x5auc_classification3d_110.pth"

##Preprocessing for segmentation
- Input: CT files in italy_ds/baseline, mask in italy_ds/mask
- Output: brain window _brain.nii.gz and groundtruth _ich_seg.nii.gz with size (512x512x48) in folder italy_ds/preprocessing
Code extract brain window, skull stripping and remove small objects, boundary. - ResizeWithPadOrCrop to (512x512x48)

In [5]:
total_start = time.time()
for _, c_row in raw_data.iterrows():
    patient = str(c_row['filename'])
    if path.exists(os.path.join(output_path, patient + "_brain.nii.gz")) == True:
        continue
    print("Processing: ", patient)
    pathCT1 = os.path.join(data_dir, baseline_path, patient + ".nii.gz")
    nii = nib.load(pathCT1)
    img1 = getbrainwindow(nii)
    spacing = nii.header['pixdim'][1:4]
    if img1.shape[2] > 48:
        new_spacing = [spacing[0], spacing[1], 5]
        img1 = resample3d(img1, spacing, new_spacing)
    else:
        new_spacing = spacing
    img1 = removeSkull(img1)
    image = resizeImgResizeWithPadOrCrop(img1, (512, 512, 48))
    empty_header = nib.Nifti1Header()
    affine = np.eye(4)
    affine[:3, :3] = np.diag(new_spacing)
    clipped_img = nib.Nifti1Image(image[0, :, :, :], affine, empty_header)
    nib.save(clipped_img, os.path.join(output_path, patient + "_brain.nii.gz"))
total_time = time.time() - total_start
print("Finish: ", total_time)

Processing:  10028
Processing:  10042
Processing:  10052
Processing:  10056
Processing:  E106342615
Finish:  26.478779315948486


##Segmentation
- Input: brain window _brain.nii.gz and groundtruth _ich_seg.nii.gz with size (512x512x48) in folder italy_ds/preprocessing
- Output: hematoma region in folder italy_ds/preprocessing/*_seg_auto.nii.gz with size (512x512x48)
- Using SegResNet model

In [ ]:
total_start = time.time()
xtest = []
nametest = []
for _, c_row in raw_data.iterrows():
    xtest = np.append(xtest, os.path.join(output_path, str(c_row['filename']) + "_brain.nii.gz"))
    nametest = np.append(nametest, str(c_row['filename']))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        EnsureTyped(keys=["image"]),
        NormalizeIntensityd(keys="image"),
        ScaleIntensityd(keys="image"),
    ]
)
num_batch = 1
test_files = [{"image": t2Img, "name": name} for t2Img, name in
              zip(xtest, nametest)]
test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=num_batch, shuffle=False,
                         num_workers=num_batch, pin_memory=torch.cuda.is_available())
model = SegResNet(
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    init_filters=16,
    in_channels=1,
    out_channels=1,
    dropout_prob=0.2,
).to(device)
model.load_state_dict(
    torch.load(os.path.join(weight_segmentation_path), map_location=torch.device('cpu')))

post_trans = Compose(
    [Activations(sigmoid=True), AsDiscrete(threshold=0.5)]
)

model.eval()
with torch.no_grad():
    num = 0
    for val_data in test_loader:
        if path.exists(os.path.join(output_path, str(val_data["name"][0]) + "_seg_auto.nii.gz"))==True:
              continue
        val_inputs = val_data["image"].to(device)
        val_outputs = inference(val_inputs)
        val_outputs = post_trans(val_outputs)
        for p in range(num_batch):
            print(num, "Patient:", val_data["name"][p])
            num = num + 1
            t1 = val_outputs[p, 0, :, :, :].cpu().numpy()
            nii = nib.load(os.path.join(output_path, str(val_data["name"][p]) + "_brain.nii.gz"))
            empty_header = nib.Nifti1Header()
            affine = np.eye(4)
            affine[:3, :3] = np.diag((nii.header["pixdim"])[1:4])
            pathCT1seg = os.path.join(output_path, str(val_data["name"][p]) + "_seg_auto.nii.gz")
            imgseg = nib.Nifti1Image(t1, affine, empty_header)
            nib.save(imgseg, pathCT1seg)
total_time = time.time() - total_start
print("Finish: ", total_time)

0 Patient: 10028
1 Patient: 10042
2 Patient: 10052
3 Patient: 10056
4 Patient: E106342615
Finish:  4228.118148803711


##Preprocessing for classification
- Input: CT files in italy_ds/baseline, hematoma region in italy_ds/preprocessing/*_seg_auto.nii.gz
- Output: brain windows and hematoma region in folder italy_ds/classification/*_brain_seg_dilation.nii.gz with size (2x128x128x96)
- Code: Get brain window from preprocessing of segmentation step, resample. Select group of slices containing Hematoma
- Resize to 2x192x192x96
- CenterSpatialCrop to 2x128x128x96

In [ ]:
total_start = time.time()
xtest = []
ytest = []
nametest = []
for _, c_row in raw_data.iterrows():
    patient = str(c_row['filename'])
    if path.exists(os.path.join(output_path_pre_classification, patient + "_brain_seg_dilation.nii.gz")) == True:
        continue
    print("Patient:" + patient)
    pathCT1seg = os.path.join(output_path, str(c_row['filename']) + "_seg_auto.nii.gz")
    nii = nib.load(pathCT1seg)
    spacing = (nii.header["pixdim"])[1:4]
    voxel_size = [1, 1, 1]
    tmpimg1seg = nii.get_fdata()
    for i in range(tmpimg1seg.shape[2]):
        if np.sum(tmpimg1seg[:, :, i]) < 50:
            tmpimg1seg[:, :, i] = 0
    tmpimg1seg = resample3d(tmpimg1seg, spacing, voxel_size)
    tmpimg1seg[tmpimg1seg > 0] = 1

    pathCT1 = os.path.join(output_path, patient + "_brain.nii.gz")
    nii = nib.load(pathCT1)
    tmpimg1 = nii.get_fdata()
    tmpimg1 = resample3d(tmpimg1, spacing, voxel_size)

    img1, img1seg = selectGroupSlice(tmpimg1, tmpimg1seg)
    img1dilation = getDilation(img1, img1seg)

    image = np.zeros([2, img1.shape[0], img1.shape[1], img1.shape[2]])
    image[0, :, :, :] = img1
    image[1, :, :, :] = img1dilation
    resizeImage = Resize([192, 192, 96])
    cropImage = CenterSpatialCrop(roi_size=(128, 128, 96))
    scaleImage = ScaleIntensity()
    image = resizeImage(image)
    image = cropImage(image)
    image = scaleImage(image)
    # save file
    pathCT1clipseg = os.path.join(data_dir, "classification", patient + "_brain_seg_dilation.nii.gz")
    empty_header = nib.Nifti1Header()
    nii = nib.load(pathCT1)
    clipped_img = nib.Nifti1Image(image, nii.affine, empty_header)
    nib.save(clipped_img, pathCT1clipseg)
total_time = time.time() - total_start
print("Finish: ", total_time)

Finish:  0.040064096450805664


##Classification
- Input: brain windows and hematoma region in folder italy_ds/classification/*_brain_seg_dilation.nii.gz (2x128x128x96)
- Output: AUC
- Code: using MyDenseNet121 which is developed from DenseNet121


In [ ]:
total_start = time.time()
pathFilenames = []
labels = []
names = []
for _, c_row in raw_data.iterrows():
    pathFilenames.append(os.path.join(data_dir, str(c_row['filename']) + "_brain_seg_dilation.nii.gz"))
    labels.append(c_row['label'])
    names.append(str(c_row['filename']))
labels = np.asarray(labels).astype(int)
pathFilenames = np.asarray(pathFilenames)
names = np.asarray(names)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MyDenseNet121(spatial_dims=3, in_channels=2, out_channels=128).to(device)
model.load_state_dict(torch.load(weight_classification_path, map_location=torch.device('cpu')))
model.eval()
xval = pathFilenames
yval = labels
val_ds = ImageDataset(
    image_files=xval, labels=yval)
val_loader = DataLoader(val_ds, batch_size=10, shuffle=False,
                        num_workers=10, pin_memory=torch.cuda.is_available())
y_pred_trans = Compose([EnsureType(), Activations(sigmoid=True)])
y_trans = Compose([EnsureType()])
with torch.no_grad():
    y_pred = torch.tensor([], dtype=torch.float32, device=device)
    y = torch.tensor([], dtype=torch.double, device=device)
    for val_data in val_loader:
        inputs, val_labels = val_data[0].to(
            device), val_data[1].to(device)
        val_outputs = model(inputs)
        y_pred1 = val_outputs.flatten()
        y1 = val_labels
        y_pred = torch.cat([y_pred, y_pred1], dim=0)
        y = torch.cat([y, y1], dim=0)
    y_onehot = [y_trans(i) for i in decollate_batch(y)]
    y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
    auc_metric1 = monai.metrics.ROCAUCMetric()
    auc_metric1(y_pred_act, y_onehot)
    print("AUC total= ", auc_metric1.aggregate())
    auc_metric1.reset()
total_time = time.time() - total_start
print("Finish: ", total_time)
